In [1]:
from __future__ import division, print_function

import argparse
import os

import numpy as np
import pandas as pd
import keras
from keras import backend as K
from keras.models import Model
from keras.utils import get_custom_objects
from tqdm import tqdm

import NCI60

Using TensorFlow backend.


Importing candle utils for keras


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

if K.backend() == 'tensorflow' and 'NUM_INTRA_THREADS' in os.environ:
    import tensorflow as tf
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

    # Uncommit when running on an optimized tensorflow where NUM_INTER_THREADS and
    # NUM_INTRA_THREADS env vars are set.
    print('NUM_INTER_THREADS: ', os.environ['NUM_INTER_THREADS'])
    print('NUM_INTRA_THREADS: ', os.environ['NUM_INTRA_THREADS'])
    session_conf = tf.ConfigProto(
        inter_op_parallelism_threads=int(os.environ['NUM_INTER_THREADS']),
        intra_op_parallelism_threads=int(os.environ['NUM_INTRA_THREADS']))
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

In [16]:
class PermanentDropout(keras.layers.Dropout):
    def __init__(self, rate, **kwargs):
        super(PermanentDropout, self).__init__(rate, **kwargs)
        self.uses_learning_phase = False

    def call(self, x, mask=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(x)
            x = K.dropout(x, self.rate, noise_shape)
        return x


def get_parser(description=None):
    parser = argparse.ArgumentParser(description=description)
    parser.add_argument('-s', '--sample_set',
                        default='NCI60',
                        help='cell sample set: NCI60, NCIPDM, GDSC, RTS, ...')
    parser.add_argument('-d', '--drug_set',
                        default='ALMANAC',
                        help='drug set: ALMANAC, GDSC, NCI_IOA_AOA, RTS, ...')
    parser.add_argument('-z', '--batch_size', type=int,
                        default=100000,
                        help='batch size')
    parser.add_argument('--step', type=int,
                        default=10000,
                        help='number of rows to inter in each step')
    parser.add_argument('-m', '--model_file',
                        default='uq.model.h5',
                        help='trained model file')
    parser.add_argument('-n', '--n_pred', type=int,
                        default=1,
                        help='the number of predictions to make for each sample-drug combination for uncertainty quantification')
    parser.add_argument('-w', '--weights_file',
                        default='uq.weights.h5',
                        help='trained weights file (loading model file alone sometimes does not work in keras)')
    parser.add_argument('--ns', type=int,
                        default=0,
                        help='the first n entries of cell samples to subsample')
    parser.add_argument('--nd', type=int,
                        default=0,
                        help='the first n entries of drugs to subsample')
    parser.add_argument("--si", type=int,
                        default=0,
                        help='the index of the first cell sample to subsample')
    parser.add_argument("--use_landmark_genes", action="store_true",
                        help="use the 978 landmark genes from LINCS (L1000) as expression features")
    parser.add_argument("--preprocess_rnaseq",
                        choices=['source_scale', 'combat', 'none'],
                        help="preprocessing method for RNAseq data; none for global normalization")

    return parser


def lookup(df, sample, drug1, drug2=None, value=None):
    drug2 = drug2 or drug1
    df_result = df[(df['Sample'] == sample) & (df['Drug1'] == drug1) & (df['Drug2'] == drug2)]
    if df_result.empty:
        df_result = df[(df['Sample'] == sample) & (df['Drug1'] == drug2) & (df['Drug2'] == drug1)]
    if value:
        if df_result.empty:
            return 1.0
        else:
            return df_result[value].iloc[0]
    else:
        return df_result


def custom_combo_score(combined_growth, growth_1, growth_2):
    if growth_1 <= 0 or growth_2 <= 0:
        expected_growth = min(growth_1, growth_2)
    else:
        expected_growth = growth_1 * growth_2
    custom_score = (expected_growth - combined_growth) * 100
    return custom_score


def cross_join(df1, df2, **kwargs):
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1

    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey', axis=1)
    # res.index = pd.MultiIndex.from_product((df1.index, df2.index))

    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)

    return res


def cross_join3(df1, df2, df3, **kwargs):
    return cross_join(cross_join(df1, df2), df3, **kwargs)


def prepare_data(sample_set='NCI60', drug_set='ALMANAC', use_landmark_genes=False, preprocess_rnaseq=None):
    df_expr = NCI60.load_sample_rnaseq(use_landmark_genes=use_landmark_genes, preprocess_rnaseq=preprocess_rnaseq, sample_set=sample_set)
    # df_old = NCI60.load_cell_expression_rnaseq(use_landmark_genes=True)
    # df_desc = NCI60.load_drug_descriptors_new()
    df_desc = NCI60.load_drug_set_descriptors(drug_set=drug_set)
    return df_expr, df_desc


def initialize_parameters():
    description = 'Infer drug pair response from trained combo model.'
    parser = get_parser(description)
    args = parser.parse_args("")
    return args

## Run function

In [17]:
## initialize parameters
args = initialize_parameters()


In [5]:
get_custom_objects()['PermanentDropout'] = PermanentDropout

In [6]:
model = keras.models.load_model(args.model_file, compile=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model.load_weights(args.weights_file)

In [8]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input.cell.rnaseq (InputLayer)  (None, 942)          0                                            
__________________________________________________________________________________________________
input.drug1.descriptors (InputL (None, 3839)         0                                            
__________________________________________________________________________________________________
input.drug2.descriptors (InputL (None, 3839)         0                                            
__________________________________________________________________________________________________
cell.rnaseq (Model)             (None, 1000)         2945000     input.cell.rnaseq[0][0]          
____________________________________________________________________________________________

In [18]:
df_expr, df_desc = prepare_data(sample_set=args.sample_set, drug_set=args.drug_set, use_landmark_genes=True)

In [19]:
print('total available samples: ', df_expr[['Sample']].shape[0])
print('total available drugs: ', df_desc[['Drug']].shape[0])

total available samples:  60
total available drugs:  104


In [25]:
df_expr.head()

,Sample,rnaseq.AARS,rnaseq.ABCB6,rnaseq.ABCC5,rnaseq.ABCF1,rnaseq.ABCF3,rnaseq.ABHD4,rnaseq.ABHD6,rnaseq.ABL1,rnaseq.ACAA1,...,rnaseq.ZMIZ1,rnaseq.ZMYM2,rnaseq.ZNF131,rnaseq.ZNF274,rnaseq.ZNF318,rnaseq.ZNF395,rnaseq.ZNF451,rnaseq.ZNF586,rnaseq.ZNF589,rnaseq.ZW10
0,NCI60.786-0,0.111146,-0.076674,-0.799897,0.734616,0.443240,0.214665,0.209361,1.349099,0.906567,...,0.803903,-1.221608,0.517746,-0.302476,0.354421,1.659058,0.267895,-1.163664,-1.190880,0.275546
1,NCI60.A498,-0.759206,0.909713,-0.124898,-0.327623,-0.046552,2.043863,-1.062378,0.753049,-0.691236,...,0.923456,-0.952465,-0.386645,-0.594410,0.103503,1.544112,-0.845434,0.308164,-1.467829,-1.205671
2,NCI60.A549,1.016783,1.168438,-1.502447,0.412363,-0.086265,1.397770,0.261269,0.741361,0.580134,...,1.064746,-1.131894,-0.744195,0.184081,-0.633566,-0.473834,-2.213900,0.689262,-0.389186,-0.023689
3,NCI60.ACHN,-0.159369,-0.893273,-0.248877,-1.497279,-1.899822,0.265010,-1.049401,-0.555924,-1.911064,...,-0.869839,0.288581,-2.027168,-0.813360,-2.029294,-1.086881,-1.262932,-0.730000,-0.360033,-1.908875
4,NCI60.BT-549,0.452230,-0.246462,-0.731020,0.663004,-1.145277,0.181102,-0.322693,1.769840,-0.192996,...,0.086585,0.049343,0.791167,1.728897,0.040774,0.905521,0.035951,-0.572304,-0.651559,0.066081


In [23]:
df_desc.head()

,Drug,dragon7.MW,dragon7.AMW,dragon7.Sv,dragon7.Se,dragon7.Sp,dragon7.Si,dragon7.Mv,dragon7.Me,dragon7.Mp,...,dragon7.Psychotic-80,dragon7.Psychotic-50,dragon7.Hypertens-80,dragon7.Hypertens-50,dragon7.Hypnotic-80,dragon7.Hypnotic-50,dragon7.Neoplastic-80,dragon7.Neoplastic-50,dragon7.Infective-80,dragon7.Infective-50
0,NSC.102816,-0.732376,-0.166781,-0.728085,-0.660214,-0.814719,-0.635373,-0.922250,1.447428,-1.158866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NSC.105014,-0.548597,-0.152677,-0.568929,-0.524018,-0.603171,-0.473096,-0.825042,0.419161,-0.653218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NSC.109724,-0.672515,0.559764,-0.804727,-0.853811,-0.755935,-0.835017,0.866379,0.507298,0.966727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NSC.118218,-0.554912,-0.166224,-0.568514,-0.495142,-0.656882,-0.453157,-0.805600,1.271154,-1.046500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NSC.119875,-0.464731,8.211406,-1.313754,0.000000,-1.154110,-1.314917,0.788612,0.000000,5.704834,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
if args.ns > 0 and args.si > 0:
    df_sample_ids = df_expr[['Sample']].iloc[args.si:args.si + args.ns]
elif args.si > 0:
    df_sample_ids = df_expr[['Sample']].iloc[args.si:]
elif args.ns > 0:
    df_sample_ids = df_expr[['Sample']].head(args.ns)
else:
    df_sample_ids = df_expr[['Sample']].copy()
if args.nd > 0:
    df_drug_ids = df_desc[['Drug']].head(args.nd)
else:
    df_drug_ids = df_desc[['Drug']].copy()

df_sum = cross_join3(df_sample_ids, df_drug_ids, df_drug_ids, suffixes=('1', '2'))

n_samples = df_sample_ids.shape[0]
n_drugs = df_drug_ids.shape[0]
n_rows = n_samples * n_drugs * n_drugs

In [29]:
df_sum.head()

,Sample,Drug1,Drug2,N
0,NCI60.786-0,NSC.102816,NSC.102816,1
1,NCI60.786-0,NSC.102816,NSC.105014,1
2,NCI60.786-0,NSC.102816,NSC.109724,1
3,NCI60.786-0,NSC.102816,NSC.118218,1
4,NCI60.786-0,NSC.102816,NSC.119875,1


In [21]:
print('Predicting drug response for {} combinations: {} samples x {} drugs x {} drugs'.format(n_rows, n_samples, n_drugs, n_drugs))
n = args.n_pred
df_sum['N'] = n
df_seq = pd.DataFrame({'Seq': range(1, n + 1)})
df_all = cross_join(df_sum, df_seq)

Predicting drug response for 648960 combinations: 60 samples x 104 drugs x 104 drugs
